In [92]:
import pandas as pd
import numpy as np

In [93]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.4f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Overview
1. Identify US users 
2. Identify US Tweets 
3. Delete all obs. without US User or Tweet 
4. Match to list of users we collected

## Import GeoCov19 Data

In [94]:
# df = pd.read_json(r"C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/geo_feb_01_10/geo_2020-02-01/geo_2020-02-01.json", lines=True)

# df = np.array_split(df, num_chunks)[chunk_num]
# df = df.iloc[:5000]

In [95]:
# for chunk in df:
#     print(pd.read_json(chunk).head())

# from io import StringIO

# with pd.read_json(StringIO(r"C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/geo_feb_01_10/geo_2020-02-01/geo_2020-02-01.json"), lines=True, chunksize=100000) as reader:
#     reader
#     for chunk in reader:
#         print(chunk.head())

### Make Uppercase

In [96]:
# def make_dict_uppercase(d):
#     '''
#     input = dictionary
#     output = uppercase dictionary
#     '''
    
#     return {key.upper(): value.upper() for key,value in d.items()}

# df['user_location'] = df['user_location'].apply(make_dict_uppercase)


# ##################################################################################


# def make_list_of_dicts_uppercase(list_of_dicts):
#     '''
#     input = list of dictionaries
#     output = uppercase list of dictionaries
#     '''
    
#     return [make_dict_uppercase(d) for d in list_of_dicts]

In [97]:
# df['tweet_locations'] = df['tweet_locations'].apply(make_list_of_dicts_uppercase)
# df['user_location'] = df['user_location'].apply(make_dict_uppercase)

## Get (State & County) --> FIPS dictionary

In [98]:
# # import data from wikipedia
# fips = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county")[1]

# # remove all hyperlinks (these look like "... County [h]", etc.)
# fips['County or equivalent'] = fips['County or equivalent'].str.replace(r"\[.*\]","")

# # convert to uppercase
# fips['County or equivalent'] = fips['County or equivalent'].apply(lambda x: x.upper())
# fips['State or equivalent'] = fips['State or equivalent'].apply(lambda x: x.upper())

# # replace "St." with "Saint"
# fips['County or equivalent'] = [x.replace('ST.','SAINT') for x in fips['County or equivalent']]

# # remove everything after a comma in a county name (e.g. "ANCHORAGE, MUNICIPALITY OF")
# fips['County or equivalent'] = [x.split(',')[0] for x in fips['County or equivalent']]

# # replace DC info to correspond to GeoCov19 format
# dc_loc = fips[fips['County or equivalent']=='DISTRICT OF COLUMBIA'].index.tolist()[0]
# fips['State or equivalent'].loc[dc_loc] = 'WASHINGTON, D.C.'
# fips['County or equivalent'].loc[dc_loc] = 'WASHINGTON'

# ########################################################################

# # create dictionary
# state_fips_dict = {k: f.groupby('County or equivalent')['FIPS'].apply(list).to_dict()
#      for k, f in fips.groupby('State or equivalent')}

# # clean dictionary
# for state in state_fips_dict:
#     for county in state_fips_dict[state]:
#         state_fips_dict[state][county] = state_fips_dict[state][county][0]

# Map county names to FIPS (using FIPS dict)

### Get FIPS from a single dictionary

In [99]:
# def get_fips_from_loc(loc):
#     '''
#     input = tweet_location from GeoCov19 data (single dictionary)
#     output = FIPS code corresponding to counties mentioned
#     '''
#     try:
       
#         if loc['COUNTRY_CODE']=='US':

#             try:

#                 if loc['COUNTY'].split(" ")[-1] in ('COUNTY', 'PARISH'):

#                     return state_fips_dict[ loc['STATE'] ] [loc['COUNTY'] ]

#                 elif 'COUNTY' in loc:

#                     try:

#                         return state_fips_dict[ loc['STATE'] ] [loc['COUNTY'] + " " + "COUNTY"]

#                     except Exception as e:

#                         pass

#             except Exception as e:

#                 pass
#     except:
        
#         pass

### Get FIPS from a list of dictionaries

In [100]:
# def get_fips_from_list_of_locs(tweet_locs):
#     '''
#     input = tweet_locations from GeoCov19 data (list of dicts)
#     output = list of FIPS codes corresponding to counties mentioned
#     '''

#     temp_list = [get_fips_from_loc(loc) for loc in tweet_locs]
    
#     return [x for x in temp_list if x is not None]

## NEED TO INSPECT COUNTY MAPPINGS TO ENSURE THAT IT PICKS UP ON STRINGS WELL
May have to implement fuzzy-matching if errors continue? Probably easiest/best to just find errors, since they won't change

# ALL REPLACEMENTS MUST BE DONE IN "fips"
* ~"Pointe Coupee Parish County" should be "Pointe Coupee Parish"~
* ~Need to replace "St." with "Saint" in "fips"~
* Fix counties that aren't cross-referenced correctly:
    * "D.C."
    * "SAN FRANCISCO"
    * There are probably more

In [101]:
# df['Tweet FIPS'] = df['tweet_locations'].apply(get_fips_from_list_of_locs)

# df['User FIPS'] = df['user_location'].apply(get_fips_from_loc)

## Percentage of GeoCov19 Tweets with county-level user data

In [102]:
# df[~df['User FIPS'].isnull()]['geo_source'].value_counts() / len(df) * 100

## Keep this

In [103]:
# def aggregate_by_user(df):
#     '''
#     Input = GeoCov19 dataframe
#     Output = pd.DatFrame with four columns: 'user_id'; 'User FIPS - user_location', 'User FIPS - place', 'User FIPS - coordinates'
#     '''
    
#     loc_type_dict = {}
    
#     for loc_source in ['user_location', 'place', 'coordinates']:

#         loc_type_dict[loc_source] = df[ df['geo_source']==loc_source ].groupby(by='user_id').agg({'User FIPS': set})        
    
#         loc_type_dict[loc_source]['User FIPS'] = loc_type_dict[loc_source]['User FIPS'].apply(list)
#     # merging
    
#     grouped_df = loc_type_dict['user_location']
    
#     grouped_df.columns = grouped_df.columns + ' - user_location'
    
#     grouped_df = grouped_df.merge(loc_type_dict['place'], on='user_id', suffixes=(None, f" - place"), how='outer')
    
#     grouped_df = grouped_df.merge(loc_type_dict['coordinates'], on='user_id', suffixes=(' - place', f" - coordinates"), how='outer')
        
#     grouped_df.columns = str(df['Date'].unique()[0]) + ' - ' + grouped_df.columns
    
#     return grouped_df.reset_index()

In [104]:
# df['Date'] = pd.to_datetime(df['created_at'], format='%Y-%m-%d').dt.date

In [105]:
# aggregate_by_user(df[~df['User FIPS'].isnull()].head(20))

# FINAL CODE STARTS HERE

In [106]:
import pandas as pd
import numpy as np

In [107]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.4f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Uppercase

In [108]:
def make_dict_uppercase(d):
    '''
    input = dictionary
    output = uppercase dictionary
    '''
    
    return {key.upper(): value.upper() for key,value in d.items()}

##################################################################################


def make_list_of_dicts_uppercase(list_of_dicts):
    '''
    input = list of dictionaries
    output = uppercase list of dictionaries
    '''
    
    return [make_dict_uppercase(d) for d in list_of_dicts]

## Get (State & County) --> FIPS dictionary

In [109]:
# import data from wikipedia
fips = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county")[1]

# remove all hyperlinks (these look like "... County [h]", etc.)
fips['County or equivalent'] = fips['County or equivalent'].str.replace(r"\[.*\]","")

# convert to uppercase
fips['County or equivalent'] = fips['County or equivalent'].apply(lambda x: x.upper())
fips['State or equivalent'] = fips['State or equivalent'].apply(lambda x: x.upper())

# replace "St." with "Saint"
fips['County or equivalent'] = [x.replace('ST.','SAINT') for x in fips['County or equivalent']]

# remove everything after a comma in a county name (e.g. "ANCHORAGE, MUNICIPALITY OF")
fips['County or equivalent'] = [x.split(',')[0] for x in fips['County or equivalent']]

# replace DC info to correspond to GeoCov19 format
dc_loc = fips[fips['County or equivalent']=='DISTRICT OF COLUMBIA'].index.tolist()[0]
fips['State or equivalent'].loc[dc_loc] = 'WASHINGTON, D.C.'
fips['County or equivalent'].loc[dc_loc] = 'WASHINGTON'

############################################################################################

# Save FIPS dataframe to file

FIPS_SAVE_PATH = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Geo Cross Referencing/County to FIPS.csv"

fips.to_csv(FIPS_SAVE_PATH, index=False)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [110]:
fips = pd.read_csv(FIPS_SAVE_PATH)

In [111]:
# create state:{county: fips} nested dictionary
state_fips_dict = {k: f.groupby('County or equivalent')['FIPS'].apply(list).to_dict()
     for k, f in fips.groupby('State or equivalent')}

# clean dictionary
for state in state_fips_dict:
    for county in state_fips_dict[state]:
        state_fips_dict[state][county] = state_fips_dict[state][county][0]

# Map county names to FIPS (using FIPS dict)

### Fuzzy Matching Function

In [112]:
def fuzzy_match_counties(county, counties_dict):
    '''
    input = county we're searching for; dict of counties we're trying to match it to and their correspodning FIPS
    output = name of the county that it's closest to
    '''
    # create list of jaro-winkler similarities between the misspelled county and all counties in the state
    jaro_distances = np.array([jellyfish.jaro_winkler_similarity(county, county_from_list) for county_from_list in list(counties_dict.keys())])
        
    # return county with smallest jaro-winkler distance
    return counties_dict[list(counties_dict.keys())[np.argmin(jaro_distances)]]

### Get FIPS from a single dictionary

In [113]:
def get_fips_from_loc(loc):
    '''
    input = tweet_location from US Election data: (STATE, COUNTY) tuple
    output = FIPS code corresponding to county 
    '''
    try:
        if loc['COUNTRY_CODE']=='US':
            try:
                return state_fips_dict[ loc['STATE'] ][ loc['COUNTY'] ]
            except:
#                 print('not exact match')
#                 print(loc['STATE'])
#                 print(state_fips_dict[ loc['STATE'] ])
                try:
                    return fuzzy_match_counties(loc['COUNTY'], state_fips_dict[loc['STATE']])
                except Exception as e:
                    return np.nan
        else:
            return np.nan
    except:
#         print('not US')
        return np.nan

In [114]:
# def get_fips_from_loc(loc):
#     '''
#     input = tweet_location from GeoCov19 data (single dictionary)
#     output = FIPS code corresponding to counties mentioned
#     '''
#     try:
       
#         if loc['COUNTRY_CODE']=='US':

#             try:

#                 if loc['COUNTY'].split(" ")[-1] in ('COUNTY', 'PARISH'):

#                     return state_fips_dict[ loc['STATE'] ] [loc['COUNTY'] ]

#                 elif 'COUNTY' in loc:

#                     try:

#                         return state_fips_dict[ loc['STATE'] ] [loc['COUNTY'] + " " + "COUNTY"]

#                     except Exception as e:

#                         pass

#             except Exception as e:

#                 pass
#     except:
        
#         pass

## Aggregate Locations to Individual Users

In [115]:
def aggregate_by_user(df):
    '''
    Input = GeoCov19 dataframe
    Output = pd.DatFrame with four columns: 'user_id'; 'User FIPS - user_location', 'User FIPS - place', 'User FIPS - coordinates'
    '''
    
    loc_type_dict = {}
    
    for loc_source in ['user_location', 'place', 'coordinates']:

        loc_type_dict[loc_source] = df[ df['geo_source']==loc_source ].groupby(by='user_id').agg({'User FIPS': set})        
    
        loc_type_dict[loc_source]['User FIPS'] = loc_type_dict[loc_source]['User FIPS'].apply(list)
    # merging
    
    grouped_df = loc_type_dict['user_location']
    
    grouped_df.columns = grouped_df.columns + ' - user_location'
    
    grouped_df = grouped_df.merge(loc_type_dict['place'], on='user_id', suffixes=(None, f" - place"), how='outer')
    
    grouped_df = grouped_df.merge(loc_type_dict['coordinates'], on='user_id', suffixes=(' - place', f" - coordinates"), how='outer')
        
    grouped_df.columns = str(df['Date'].unique()[0]) + ' - ' + grouped_df.columns
    
    return grouped_df.reset_index()

## Import All CT Users

In [116]:
MASTER_USER_LIST = pd.read_csv(r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/MASTER LIST - All CT Link and Hashtag Users/All CT Link and Hashtag Users.csv")

# FINAL FUNCTION


In [118]:
def final_func(IMPORT_PATH, EXPORT_PATH):#, num_chunks, chunk_num):
    '''
    num_chunks = number of chunks to split dataframe into
    chunk_num = current chunk being worked on (should be iterating here).
                chunk_num is in [0,...,num_chunks-1]
    '''
    
    df = pd.read_json(fr"{IMPORT_PATH}", lines=True)
    
#     df = np.array_split(df, num_chunks)[chunk_num]
    
    # drop if not in master list of users
    user_intersection = np.intersect1d(df['user_id'].unique(), MASTER_USER_LIST['ID'].unique(), assume_unique=True)
    df = df.set_index('user_id').loc[user_intersection]
    
    # make uppercase
    df['user_location'] = df['user_location'].apply(make_dict_uppercase)
    #df['tweet_locations'] = df['tweet_locations'].apply(make_list_of_dicts_uppercase)
    
    # convert location strings to FIPS
    df['User FIPS'] = df['user_location'].apply(get_fips_from_loc)
    #df['Tweet FIPS'] = df['tweet_locations'].apply(get_fips_from_list_of_locs)
    
    # only keep obs with user_location
    df = df[~df['User FIPS'].isnull()]
    
    # convert date to better format
    df['Date'] = pd.to_datetime(df['created_at'], format='%Y-%m-%d').dt.date
    del df['created_at']
    
    # aggregate by user
    df = aggregate_by_user(df)
            
#     return df
    df.to_csv(fr"{EXPORT_PATH}.csv")

In [ ]:
test = final_func(IMPORT_PATH='C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/geo_feb_01_10/geo_2020-02-01/geo_2020-02-01.json',
                 EXPORT_PATH=None)

# Implementing Final Function (Final Loop)

In [119]:
month_length_dict = {'feb':29, 'march':31, 'april':30}
month_number_dict = {'feb':2, 'march':3, 'april':4}

processed_geocov_files = []

# NUM_CHUNKS = 5

IMPORT_PATH_ROOT = "C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/"

EXPORT_PATH_ROOT = "C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/Intermediate Cleaned Data/"

for month in ['feb', 'march', 'april']:
    
    days = month_length_dict[month]
    month_number = month_number_dict[month]
    
    for i in range(1, days + 1):
#     for i in range(26, days + 1):
        
#         for chunk_num in range(0, NUM_CHUNKS):
        
        if i < 10:

            IMP_PATH = IMPORT_PATH_ROOT + fr"geo_{month}_01_10/geo_2020-0{month_number}-0{i}/geo_2020-0{month_number}-0{i}.json"
            EXP_PATH = EXPORT_PATH_ROOT + f"2020-{month_number}-{i} - SPLIT {chunk_num} - TEST.csv"
            processed_geocov_files.append(EXP_PATH)

            print(i, '\n', IMP_PATH, '\n', EXP_PATH)
            final_func(IMP_PATH, EXP_PATH)#, NUM_CHUNKS, chunk_num)

        elif i == 10:
            IMP_PATH = IMPORT_PATH_ROOT + fr"geo_{month}_01_10/geo_2020-0{month_number}-{i}/geo_2020-0{month_number}-{i}.json"
            EXP_PATH = EXPORT_PATH_ROOT + f"2020-{month_number}-{i} - SPLIT {chunk_num}.csv"
            processed_geocov_files.append(EXP_PATH)

            print(i, '\n', IMP_PATH, '\n', EXP_PATH)
            final_func(IMP_PATH, EXP_PATH)# ,NUM_CHUNKS, chunk_num)

        elif i <= 20:

            IMP_PATH = IMPORT_PATH_ROOT + fr"geo_{month}_11_20/geo_2020-0{month_number}-{i}/geo_2020-0{month_number}-{i}.json"
            EXP_PATH = EXPORT_PATH_ROOT + f"2020-{month_number}-{i} - SPLIT {chunk_num}.csv"
            processed_geocov_files.append(EXP_PATH)

            print(i, '\n', IMP_PATH, '\n', EXP_PATH)
            final_func(IMP_PATH, EXP_PATH)# ,NUM_CHUNKS, chunk_num)            
        else:

            IMP_PATH = IMPORT_PATH_ROOT + fr"geo_{month}_21_{days}/geo_2020-0{month_number}-{i}/geo_2020-0{month_number}-{i}.json"
            EXP_PATH = EXPORT_PATH_ROOT + f"2020-{month_number}-{i} - SPLIT {chunk_num}.csv"
            processed_geocov_files.append(EXP_PATH)

            print(i, '\n', IMP_PATH, '\n', EXP_PATH)
            final_func(IMP_PATH, EXP_PATH)# ,NUM_CHUNKS, chunk_num)

1 
 C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/geo_feb_01_10/geo_2020-02-01/geo_2020-02-01.json 
 C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/Intermediate Cleaned Data/2020-2-1 - SPLIT 0 - TEST.csv
2 
 C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/geo_feb_01_10/geo_2020-02-02/geo_2020-02-02.json 
 C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/Intermediate Cleaned Data/2020-2-2 - SPLIT 0 - TEST.csv


KeyboardInterrupt: 

# Now Match to Twint Dataframe

In [2]:
from os import listdir
from os.path import isfile, join

In [28]:
path = r"C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/Intermediate Cleaned Data/"

processed_geocov_files = [f for f in listdir(path) if isfile(join(path, f))]

df = pd.read_csv(path + processed_geocov_files.pop(0))

del df['Unnamed: 0']

for file in processed_geocov_files:
    
    df = df.merge(pd.read_csv(path + file).drop(labels='Unnamed: 0', axis=1), how='outer', on='user_id')
    
# sort columns
df = df.reindex(sorted(df.columns), axis=1)

# set index to 'user ID'
df = df.set_index('user_id')

['2020-2-1.csv.csv',
 '2020-2-10.csv.csv',
 '2020-2-11.csv.csv',
 '2020-2-12.csv.csv',
 '2020-2-13.csv.csv',
 '2020-2-14.csv.csv',
 '2020-2-15.csv.csv',
 '2020-2-16.csv.csv',
 '2020-2-17.csv.csv',
 '2020-2-18.csv.csv',
 '2020-2-19.csv.csv',
 '2020-2-2.csv.csv',
 '2020-2-20.csv.csv',
 '2020-2-21.csv.csv',
 '2020-2-22.csv.csv',
 '2020-2-23.csv.csv',
 '2020-2-24.csv.csv',
 '2020-2-25.csv.csv',
 '2020-2-3.csv.csv',
 '2020-2-4.csv.csv',
 '2020-2-5.csv.csv',
 '2020-2-6.csv.csv',
 '2020-2-7.csv.csv',
 '2020-2-8.csv.csv',
 '2020-2-9.csv.csv']

In [37]:
sorted(processed_geocov_files)

['2020-2-10.csv.csv',
 '2020-2-11.csv.csv',
 '2020-2-12.csv.csv',
 '2020-2-13.csv.csv',
 '2020-2-14.csv.csv',
 '2020-2-15.csv.csv',
 '2020-2-16.csv.csv',
 '2020-2-17.csv.csv',
 '2020-2-18.csv.csv',
 '2020-2-19.csv.csv',
 '2020-2-2.csv.csv',
 '2020-2-20.csv.csv',
 '2020-2-21.csv.csv',
 '2020-2-22.csv.csv',
 '2020-2-23.csv.csv',
 '2020-2-24.csv.csv',
 '2020-2-25.csv.csv',
 '2020-2-3.csv.csv',
 '2020-2-4.csv.csv',
 '2020-2-5.csv.csv',
 '2020-2-6.csv.csv',
 '2020-2-7.csv.csv',
 '2020-2-8.csv.csv',
 '2020-2-9.csv.csv']